In [25]:
%%capture
import lightning 

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import lightning as L
from torch.utils.data import TensorDataset,DataLoader
import matplotlib.pyplot as plt

<h3>__00__</h3>
<h3>_0__0_</h3>
<h3>0____0</h3>     
<h3>0____0</h3>
<h3>_0__0_</h3>
<h3>__00__</h3>
<h3>       </h3>
<h3>0____0</h3>
<h3>_0__0_</h3>
<h3>__00__</h3>
<h3>__00__</h3>
<h3>_0__0_</h3>
<h3>0____0</h3>

In [27]:
oimg=    [[0, 0, 1, 1, 0, 0],
           [0, 1, 0, 0, 1, 0],
           [1, 0, 0, 0, 0, 1],
           [1, 0, 0, 0, 0, 1],
           [0, 1, 0, 0, 1, 0],
           [0, 0, 1, 1, 0, 0]]
ximg=      [[1, 0, 0, 0, 0, 1],
           [0, 1, 0, 0, 1, 0],
           [0, 0, 1, 1, 0, 0],
           [0, 0, 1, 1, 0, 0],
           [0, 1, 0, 0, 1, 0],
           [1, 0, 0, 0, 0, 1]]

In [28]:
xi=torch.tensor([oimg,ximg]).type(torch.float32)

In [36]:
yinp=[0,1]
yi=F.one_hot(torch.tensor(yinp)).type(torch.float32)

In [37]:
dataset=TensorDataset(xi,yi)
dl=DataLoader(dataset)

In [48]:
class Net(L.LightningModule):
    def __init__(self):
        super().__init__()
        L.seed_everything(seed=42)
        self.convolve=nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3)
        self.maxpooling=nn.MaxPool2d(kernel_size=2,stride=2)
        self.pooltohidden=nn.Linear(in_features=4,out_features=1)
        self.hiddentoout=nn.Linear(in_features=1,out_features=2)
        self.loss=nn.CrossEntropyLoss()

    def forward(self,x):
        feat=self.convolve(x)
        feat=F.relu(feat)
        pool=self.maxpooling(feat)
        pool=torch.flatten(pool,1)
        xi=self.pooltohidden(pool)
        yi=F.relu(xi)
        out=self.hiddentoout(yi)
        return out
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)

    def training_step(self,batch,batch_idx):
        x,y=batch
        out=self.forward(x)
        loss=self.loss(out,y)
        return loss
        

In [49]:
model=Net()


In [54]:
trainer=L.Trainer(max_epochs=100)
trainer.fit(model, train_dataloaders=dl)

Training: |          | 0/? [00:00<?, ?it/s]

In [55]:
for batch_num, (image, label) in enumerate(dl):
    
    ## First, run the image through the model to make a prediction
    prediction = model(image)
    
    ## Now make the prediction easy to read and interpret by
    ## running it through torch.softmax() and torch.round()
    predicted_label = torch.round(torch.softmax(prediction, dim=1), ## dim=0 applies softmax to rows, dim=1 applies soft to columns
                                  decimals=2) 
    
    ## Now print out the the predicted label and the original label
    ## so we see how well our CNN performed.
    print("predicted_label:", predicted_label)
    print("original label:", label)
    print("\n")

predicted_label: tensor([[0.8500, 0.1500]], grad_fn=<RoundBackward1>)
original label: tensor([[1., 0.]])


predicted_label: tensor([[0., 1.]], grad_fn=<RoundBackward1>)
original label: tensor([[0., 1.]])




In [56]:
testx = [[0, 1, 0, 0, 0, 0],
                   [0, 0, 1, 0, 0, 1],
                   [0, 0, 0, 1, 1, 0],
                   [0, 0, 0, 1, 1, 0],
                   [0, 0, 1, 0, 0, 1],
                   [0, 1, 0, 0, 0, 0]]
testx

[[0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 1, 0],
 [0, 0, 0, 1, 1, 0],
 [0, 0, 1, 0, 0, 1],
 [0, 1, 0, 0, 0, 0]]

In [57]:
prediction = model(torch.tensor([testx]).type(torch.float32))
predicted_label = torch.round(torch.softmax(prediction, dim=1), decimals=2)
predicted_label

tensor([[0.0500, 0.9500]], grad_fn=<RoundBackward1>)

In [58]:
# since x is[0.0,1.0] and o is [1.0,0.1] , the prediction is good

In [59]:
testo = [[0, 1, 1, 0, 0, 0],
                   [1, 0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1, 0],
                   [1, 0, 0, 1, 0, 0],
                   [0, 1, 1, 0, 0, 0]]
testo

[[0, 1, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0]]

In [60]:
prediction = model(torch.tensor([testo]).type(torch.float32))
predicted_label = torch.round(torch.softmax(prediction, dim=1), decimals=2)
predicted_label

tensor([[0.3800, 0.6200]], grad_fn=<RoundBackward1>)